# start

In [4]:
trntmstmp=1584412344
valtmstmp=1585069785
import datetime
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (trntmstmp, valtmstmp)])

grand_total=1.5e8
MIN_TM_TRN=1580947200
MIN_TM_TST=1581552000
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (MIN_TM_TRN, MIN_TM_TST)])


CHNKSZ=1e6
POST_RATE_WANTED=0.1

['2020-03-17 02:32:24', '2020-03-24 17:09:45']
['2020-02-06 00:00:00', '2020-02-13 00:00:00']


# setup

In [7]:
from pathlib import Path
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, precision_recall_curve
import pickle
from tqdm import tqdm
from collections import Counter
import category_encoders as ce
pd.set_option('display.max_rows', 500)

from functools import reduce
import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'
# p_out=f'{HOME}/output/{PRFX}'
# Path(p_out).mkdir(exist_ok=True, parents=True)

from sklearn.metrics import precision_recall_curve, auc, log_loss

def compute_prauc(pred, gt):
    prec, recall, thresh = precision_recall_curve(gt, pred)
    prauc = auc(recall, prec)
    return prauc

def calculate_ctr(gt):
    positive = len([x for x in gt if x == 1])
    ctr = positive/float(len(gt))
    return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

# https://towardsdatascience.com/how-to-calibrate-undersampled-model-scores-8f3319c1ea5b
# How to use the function?
# Let’s say your goal is to generate a model that shows the credit default probabilities and your original 
# training data has 50,000 rows with only 500 of them labeled as target class. When you sample your non-target 
# instances randomly and reduce the total row count to 10,000, while conserving 500 target rows, our calibration
# function becomes:
# calibration(model_results, 50000, 500, 10000, 500)
# Here model_results is your model probability output array. After you train your model and put the results in it, your function is ready to use. 

def calibration(data, train_pop, target_pop, sampled_train_pop, sampled_target_pop):
    calibrated_data = \
    ((data * (target_pop / train_pop) / (sampled_target_pop / sampled_train_pop)) /
    ((
        (1 - data) * (1 - target_pop / train_pop) / (1 - sampled_target_pop / sampled_train_pop)
     ) +
     (
        data * (target_pop / train_pop) / (sampled_target_pop / sampled_train_pop)
     )))

    return calibrated_data

cols=[
'toks',
'hshtgs',
'twtid',
'media',
'links',
'domns',
'twttyp',
'lang',
'tm',

'u1id',
'u1_fllwer_cnt',
'u1_fllwng_cnt',
'u1_vrfed',
'u1_create_tm',

'u2id',
'u2_fllwer_cnt',
'u2_fllwng_cnt',
'u2_vrfed',
'u2_create_tm',

'u1_fllw_u2',
'reply_tm',
'retwt_tm',
'retwt_cmmnt_tm',
'like_tm',
]
cols_cat = ['twttyp','lang']
cols_val = cols[:-4]
cols_tgt_tmstmp=[
    'retwt_tm',
    'reply_tm',
    'like_tm',
    'retwt_cmmnt_tm',
]
cols_tgt=[o.split('_tm')[0] for o in cols_tgt_tmstmp]
tgts             = ['Retweet','Reply','Like','RTwCmnt',]
assert cols_tgt == ['retwt',  'reply','like','retwt_cmmnt',]
ntgts=len(tgts)


tgt2col=dict(zip(tgts,cols_tgt))
tgt2col

{'Retweet': 'retwt',
 'Reply': 'reply',
 'Like': 'like',
 'RTwCmnt': 'retwt_cmmnt'}

In [14]:
%%time
dftrn=pd.read_csv(
    f'{p_in}/trn_{trntmstmp}.tsv',
    sep='\x01', header=None, 
    encoding='utf-8', 
    names=cols, 
    nrows=1e6
)

CPU times: user 6.99 s, sys: 1.24 s, total: 8.23 s
Wall time: 12.3 s


In [15]:
dftrn.shape

(1000000, 24)

In [16]:
dftrn.columns

Index(['toks', 'hshtgs', 'twtid', 'media', 'links', 'domns', 'twttyp', 'lang',
       'tm', 'u1id', 'u1_fllwer_cnt', 'u1_fllwng_cnt', 'u1_vrfed',
       'u1_create_tm', 'u2id', 'u2_fllwer_cnt', 'u2_fllwng_cnt', 'u2_vrfed',
       'u2_create_tm', 'u1_fllw_u2', 'reply_tm', 'retwt_tm', 'retwt_cmmnt_tm',
       'like_tm'],
      dtype='object')

In [17]:
dftrn.head()

,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,u2id,u2_fllwer_cnt,u2_fllwng_cnt,u2_vrfed,u2_create_tm,u1_fllw_u2,reply_tm,retwt_tm,retwt_cmmnt_tm,like_tm
0,101\t6417\t3410\t3398\t3184\t1909\t56910\t1683...,NaN,D4D1EBDE74F74C5DA529959AF979625C,NaN,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581258144,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,...,0000006C3074607050F1339DDCB890BB,27428,600,False,1520948869,False,NaN,NaN,NaN,NaN
1,101\t14120\t131\t120\t120\t188\t119\t11170\t12...,NaN,BFB529DAB6D384EB83E899A72AB3830D,Photo,NaN,NaN,TopLevel,B9175601E87101A984A50F8A62A1C374,1581018910,1F30A85E612A24B426013978ADB7AE46,...,000005C520010F8917EEAB6F5B6EC1C4,17,77,False,1569692352,True,NaN,NaN,NaN,NaN
2,101\t62342\t10858\t54439\t19571\t22480\t7831\t...,NaN,519078C7834E9642508F72A6C2D0F3B7,NaN,788ABAA22F1C4A9D186510C1E3E81F58,892B6E0BC037800B59112CFCB95E3301,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581244866,9E1FD934C376903895DEEBCC0F6AB920,...,000006829BEADA9EEA695CF0C334B426,2,29,False,1568107028,False,NaN,NaN,NaN,1.581247e+09
3,101\t58955\t10898\t103305\t1901\t16181\t7168\t...,F0F2FBE57F08E7C4326682B5EDA63E3E\tE78674D32346...,52AAE9E33EFAC8C478C57B31A9E31ED1,NaN,C06B84A10A75AE4BF4A102AB45CC2ABF,C0F5328C1D36CD4B34D1808012E18D46,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581250124,BEB3C337B266E2C4769065AD64724329,...,000006829BEADA9EEA695CF0C334B426,3,30,False,1568107028,False,NaN,NaN,NaN,NaN
4,101\t2435\t5656\t2594\t8279\t8623\t1925\t64126...,NaN,89C1298C55EB3D68E2784F0BFB69E6F8,NaN,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1580958805,9E1FD934C376903895DEEBCC0F6AB920,...,000006829BEADA9EEA695CF0C334B426,3,30,False,1568107028,False,NaN,NaN,NaN,NaN


In [35]:
n=np.random.choice(len(dftrn))
np.argsort(dftrn.tm)[n:n+10]

773297    759188
773298    818697
773299    125038
773300    526756
773301      1015
773302    615195
773303      6926
773304    898816
773305    819908
773306    200654
Name: tm, dtype: int64

In [49]:
n=np.random.choice(len(dftrn))
dftrn.lang[n:n+10]

266334    167115458A0DBDFF7E9C0C53A83BAC9B
266335    D3164C7FBCF2565DDF915B1B3AEFB1DC
266336    22C448FF81263D4BAF2A176145EE9EAD
266337    22C448FF81263D4BAF2A176145EE9EAD
266338    22C448FF81263D4BAF2A176145EE9EAD
266339    D3164C7FBCF2565DDF915B1B3AEFB1DC
266340    D3164C7FBCF2565DDF915B1B3AEFB1DC
266341    D3164C7FBCF2565DDF915B1B3AEFB1DC
266342    FA3F382BC409C271E3D6EAF8BE4648DD
266343    22C448FF81263D4BAF2A176145EE9EAD
Name: lang, dtype: object

In [70]:
n=np.random.choice(len(dftrn))
dftrn.twttyp[n:n+10]

258553     Retweet
258554     Retweet
258555     Retweet
258556     Retweet
258557     Retweet
258558    TopLevel
258559    TopLevel
258560    TopLevel
258561    TopLevel
258562    TopLevel
Name: twttyp, dtype: object

In [73]:
dftrn.u1id.value_counts().value_counts().sort_index()

1       459596
2        54719
3        17941
4         8504
5         5173
6         3257
7         2333
8         1766
9         1334
10        1065
11         854
12         717
13         600
14         531
15         431
16         356
17         373
18         281
19         276
20         244
21         189
22         183
23         186
24         155
25         119
26         129
27         102
28         113
29         103
30          70
31          80
32          78
33          82
34          76
35          72
36          66
37          50
38          53
39          43
40          62
41          44
42          55
43          29
44          31
45          34
46          35
47          34
48          34
49          33
50          23
51          21
52          20
53          23
54          26
55          27
56          24
57          19
58          14
59          10
60          19
61          19
62          21
63          22
64          17
65          15
66          13
67        

In [74]:
dftrn.u2id.value_counts().value_counts().sort_index()

1     438504
2     148456
3      43292
4      13728
5       5621
6       2600
7       1370
8        835
9        511
10       336
11       241
12       180
13       138
14        72
15        59
16        50
17        32
18        22
19        13
20         9
21         7
22         7
23         5
24         6
25         4
26         2
27         3
28         3
29         3
30         3
31         1
33         1
36         1
43         1
Name: u2id, dtype: int64

In [75]:
dftrn['u1u2']=dftrn.u1id+'_'+dftrn.u2id

In [76]:
dftrn.u1u2.value_counts().value_counts().sort_index()

1     970450
2      13242
3        775
4        111
5         33
6          9
7          7
8          1
10         1
11         1
Name: u1u2, dtype: int64

In [78]:
970450/len(dftrn)

0.97045